## Saving Samples [mchirp,q,chieff,z] from Normalising flow models to HDF5 file

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch
import time
from scipy.special import logsumexp

import sys

sys.path.append('../')
from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

from sample import sample

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


glasflow is using its own internal version of nflows


In [11]:
params = ['mchirp','q', 'chieff', 'z']
channels = ['CE','CHE','GC','NSC','SMT']
chi_b = [0.0,0.1,0.2,0.5]
alpha = [0.2,0.5,1.,2.,5.]
file_path='./models_unityweights.hdf5'

model_names, flows = read_models.get_models(file_path, channels, params, use_flows=True, device='cpu', no_bins=[5,4,4,4,4])
model_names.sort()
hyperparams = sorted(list(set([x.split('/', 1)[1] for x in model_names])))
Nhyper = np.max([len(x.split('/')) for x in hyperparams])

100%|██████████| 5/5 [00:05<00:00,  1.11s/it]


In [4]:
hyperparam_dict  = {}
hyperidx=0
while hyperidx < Nhyper:
    hyperidx_with_Nhyper = np.argwhere(np.asarray([len(x.split('/')) for x in hyperparams])>hyperidx).flatten()
    hyperparams_at_level = sorted(set([x.split('/')[hyperidx] for x in np.asarray(hyperparams)[hyperidx_with_Nhyper]]))
    hyperparam_dict[hyperidx] = hyperparams_at_level
    hyperidx += 1
all_models_at_deepest = all([len(x.split('/')[1:])==Nhyper for x in model_names])
while all_models_at_deepest==False:
    # loop until all models have the same length
    for model in model_names:
        # See number of hyperparameters in model, subtract one for channel
        Nhyper_in_model = len(model.split('/'))-1
        # loop until this model has all the hyperparam levels as well
        while Nhyper_in_model < Nhyper:
            model_names.remove(model)
            for new_hyperparam in hyperparam_dict[Nhyper_in_model]:
                # add new model name
                model_names.append(model+'/'+new_hyperparam)
            Nhyper_in_model += 1
        model_names.sort()
    # see if all models are at deepest level else repeat
    all_models_at_deepest = all([len(x.split('/')[1:])==Nhyper for x in model_names])

In [5]:
#inputs: x, data, pop_models, submodels_dict, channels, use_flows
hyperparams = list(set([x.split('/', 1)[1] for x in model_names]))
Nhyper = np.max([len(x.split('/')) for x in hyperparams])
channels = sorted(list(set([x.split('/')[0] for x in model_names])))

# construct dict that relates submodels to their index number
submodels_dict = {} #dummy index dict keys:0,1,2,3, items: particular models
ctr=0 #associates with either chi_b or alpha (0 or 1)
while ctr < Nhyper:
    submodels_dict[ctr] = {}
    hyper_set = sorted(list(set([x.split('/')[ctr] for x in hyperparams])))
    for idx, model in enumerate(hyper_set): #idx associates with 0,1,2,3,(4) keys
        submodels_dict[ctr][idx] = model
    ctr += 1

In [6]:
submodels_dict

{0: {0: 'chi00', 1: 'chi01', 2: 'chi02', 3: 'chi05'},
 1: {0: 'alpha02', 1: 'alpha05', 2: 'alpha10', 3: 'alpha20', 4: 'alpha50'}}

In [7]:
all_models = []
models = []
def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])

In [8]:
f = h5py.File(file_path, 'r')
f.visititems(find_submodels)
# get all unique models
all_models = sorted(list(set(all_models)))
f.close()

In [13]:
for chnl in channels:
    samples = read_models.read_hdf5(file_path, chnl)
    for chibid in range(4):
        for alphaid in range(5):
            if chnl == 'CE':
                samples[(chibid,alphaid)]['weight'] = np.ones(len(samples[(chibid,alphaid)]['weight']))
                print(samples)
                df = pd.DataFrame(data=samples[(chibid,alphaid)], columns=samples[(chibid,alphaid)].keys())
                df.to_hdf("models_unityweights.hdf5", key=f'{chnl}/{submodels_dict[0][chibid]}/{submodels_dict[1][alphaid]}')
            else:
                samples[(chibid)]['weight'] = np.ones(len(samples[(chibid)]['weight']))
                print(samples)
                df = pd.DataFrame(data=samples[(chibid)], columns=samples[(chibid)].keys())
                df.to_hdf("models_unityweights.hdf5", key=f'{chnl}/{submodels_dict[0][chibid]}')

{(0, 0):             mchirp         q    chieff         z         m1         m2  \
3016216  14.133764  0.900325  0.491114  1.189084  15.409309  17.115271   
3935186  23.587960  0.698814  0.006990  0.437880  22.722927  32.516402   
4989133  26.614096  0.788422  0.002801  0.816046  27.183739  34.478670   
1793837   6.522917  0.989520  0.275183  0.590430   7.532458   7.453520   
1909097  21.207803  0.875283  0.468603  0.640233  22.801761  26.050735   
...            ...       ...       ...       ...        ...        ...   
119858    8.544301  0.474577  0.632929  0.037534   6.853822  14.441951   
257295   21.354823  0.742844  0.036359  0.046054  21.188815  28.523918   
1970946  18.726059  0.941482  0.002567  0.224860  22.171029  20.873624   
5402207  18.857978  0.924672  0.003058  1.758124  22.530657  20.833468   
3650410   7.390631  0.483330  0.329374  1.515806   5.978976  12.370368   

              s1x       s1y       s1z       s2x       s2y       s2z  weight  \
3016216  0.000000 -0.00